In [1]:
# %load  8.1-1 原始save-系统.py
import baostock as bs
import pandas as pd
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn import preprocessing
import seaborn as sns
import datetime



sns.set(color_codes=True)
import matplotlib.pyplot as plt

### 一、获取数据并修改类型 ###
## 登陆系统
lg = bs.login()
# 显示登陆返回信息
print('login respond error_code:'+lg.error_code)
print('login respond  error_msg:'+lg.error_msg)

## 获取沪深A股历史K线数据
# 详细指标参数，参见“历史行情指标参数”章节；“分钟线”参数与“日线”参数不同。“分钟线”不包含指数。
# 分钟线指标：date,time,code,open,high,low,close,volume,amount,adjustflag
rs = bs.query_history_k_data_plus("sz.002025",
    "date,time,code,open,high,low,close,volume,amount,adjustflag",
    start_date=(datetime.date.today() + datetime.timedelta(days=-1)).strftime('%Y-%m-%d'),
    end_date=datetime.date.today().strftime('%Y-%m-%d'),
    frequency="5", adjustflag="3")
print('query_history_k_data_plus respond error_code:'+rs.error_code)
print('query_history_k_data_plus respond  error_msg:'+rs.error_msg)

## 打印结果集
data_list = []
while (rs.error_code == '0') & rs.next():
    # 获取一条记录，将记录合并在一起
    data_list.append(rs.get_row_data())
result = pd.DataFrame(data_list, columns=rs.fields)

## 修改result数据类型  
result = result.apply(pd.to_numeric,errors = 'ignore')  # 将dataframe中元素变为数值类型
result = result.drop(['date','code'],axis = 1)   # 删掉指定列，'date'和'code'为列名
result = result.set_index('time')                # 设置'time'列为指定行名
result.index = pd.to_datetime(result.index, format ='%Y%m%d%H%M%S%f')


### 二、数据预处理 ###
# 归一化
scaler = preprocessing.MinMaxScaler()
X_result = pd.DataFrame(scaler.fit_transform(result),columns=result.columns,index=result.index)

### 三、导入模型 ###
model=keras.models.load_model('C:/Users/23665/model_1htdq.h5')


### 四、查看模型的“认知水平” ###
X_pred = model.predict(np.array(X_result))
X_pred = pd.DataFrame(X_pred,
                      columns=X_result.columns)
X_pred.index = X_result.index

scored = pd.DataFrame(index=X_result.index)
scored['Loss_mae'] = np.mean(np.abs(X_pred-X_result), axis = 1)
plt.figure()
sns.distplot(scored['Loss_mae'],
              bins = 10,
              kde= True,
            color = 'blue')
plt.xlim([0.0,.5])


### 五、对测试集进行异常诊断 ###
X_pred = model.predict(np.array(X_result))
X_pred = pd.DataFrame(X_pred,
                      columns=X_result.columns)
X_pred.index = X_result.index


threshod = 0.36
scored = pd.DataFrame(index=X_result.index)
scored['Loss_mae'] = np.mean(np.abs(X_pred-X_result), axis = 1)
scored['Threshold'] = threshod
scored['Anomaly'] = scored['Loss_mae'] > scored['Threshold']
scored.head()



'''
scored.plot(logy=True,  figsize = (10,6),xlim = [(datetime.strptime(scored.index[0],'%Y/%m/%d').date()),
                                               (datetime.strptime(scored.index[-1],'%Y/%m/%d').date()),ylim = [1e-2,1e1], color = ['blue','red'])
'''
#datetime.strptime('2018/3/15', '%Y/%m/%d').date()
#为什么开始时间和最后时间一样？tmd
print(scored.index[0])
print(scored.index[-1])
from datetime import datetime

print(datetime.strptime(str(scored.index[0]), '%Y-%m-%d %H:%M:%S').date())

scored.plot(logy=True,  figsize = (10,6),xlim = [datetime.strptime(str(scored.index[0]), '%Y-%m-%d %H:%M:%S'),
                                               datetime.strptime(str(scored.index[-1]), '%Y-%m-%d %H:%M:%S')],ylim = [1e-2,1e1], color = ['blue','red'])

scored.plot(logy=True,  figsize = (10,6),xlim = [datetime.strptime(str(scored.index[0]), '%Y-%m-%d %H:%M:%S').date(),
                                               datetime.strptime(str(scored.index[-1]), '%Y-%m-%d %H:%M:%S')].date(),ylim = [1e-2,1e1], color = ['blue','red'])

'''
scored.plot(logy=True,  figsize = (10,6),xlim = [(scored.index[0]).strftime('%Y-%m-%d %H:%M:%S'),
                                               (scored.index[-1]).strftime('%Y-%m-%d %H:%M:%S')],ylim = [1e-2,1e1], color = ['blue','red'])
'''

Anomaly=scored["Anomaly"].sum()

print('异常点个数为：%d'%Anomaly)
if 0 <= Anomaly <= 10:
    print('风险等级：无风险')
elif 10 < Anomaly <= 20:
    print('风险等级：黄色预警')
elif 20 < Anomaly <= 30:
    print('风险等级：橙色预警')
else:
    print('风险等级：红色预警')






login success!
login respond error_code:0
login respond  error_msg:success
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success


ValueError: Found array with 0 sample(s) (shape=(0, 7)) while a minimum of 1 is required by MinMaxScaler.

In [11]:


Anomaly=scored["Anomaly"].sum()

print('异常点个数为：%d'%Anomaly)
if 0 <= Anomaly <= 10:
    print('风险等级：无风险')
elif 10 < Anomaly <= 20:
    print('风险等级：黄色预警')
elif 20 < Anomaly <= 30:
    print('风险等级：橙色预警')
else:
    print('风险等级：红色预警')


异常点个数为：0
风险等级：无风险
